# readme

### # contrastive loss를 사용할 때 주의점
- distance 를 기준으로, 거리가 가까우면 '같다'고 정의. 따라서, accuracy를 측정할때 살펴보면 아래와 같이 정의되어 있다는점에 주의하자!


In [24]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.'''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

---
---

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.model_selection import train_test_split

import glob
import os
import multiprocessing

import pickle
import time
import random

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, Activation, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [3]:
np.array([[1,2,3,4],
         [5,6,7,8]]).reshape(2,2,-1)

array([[[1, 2],
        [3, 4]],

       [[5, 6],
        [7, 8]]])

---

# data load

- train data

In [7]:
with open('../train_within_0_35_balanced/X_tain.pkl', 'rb') as f:
    X_train = pickle.load( f )
    
with open('../train_within_0_35_balanced/y_train.pkl', 'rb') as f:
    y_train = pickle.load( f )

In [8]:
print(X_train.shape, y_train.shape)

(330519, 512) (330519,)


- test_data : TEST_within_035

In [9]:
glob.glob('../test_data(common)/TEST_within_035/*')

['../test_data(common)/TEST_within_035/test_pair_035.pkl',
 '../test_data(common)/TEST_within_035/test_pair_v2_035.pkl',
 '../test_data(common)/TEST_within_035/test_label_035.pkl',
 '../test_data(common)/TEST_within_035/ReadME.txt']

In [10]:
with open('../test_data(common)/TEST_within_035/test_pair_v2_035.pkl', 'rb') as f:
    X_test_dict = pickle.load( f )
    
with open('../test_data(common)/TEST_within_035/test_label_035.pkl', 'rb') as f:
    y_test_dict = pickle.load( f )

In [11]:
len(X_test_dict)

100

In [12]:
len(y_test_dict)

100

In [13]:
# test data 100개 array로

X_test = np.array([])
y_test = np.array([])

for i,key in enumerate(X_test_dict.keys()):
    if i == 0:
        X_test = X_test_dict[key]
        y_test = y_test_dict[key]
    else:
        X_test = np.append( X_test, X_test_dict[key], axis=0 )
        y_test = np.append( y_test, y_test_dict[key], axis=0 )

In [14]:
print(X_test.shape, y_test.shape)

(2183, 512) (2183,)


---
---

### pair 데이터로 변환

- train

In [11]:
X_train.shape

(330519, 512)

In [13]:
tr_pairs = X_train.reshape(len(X_train), 2, 16 , 16)
tr_y = np.array(y_train, dtype=float)

In [14]:
tr_y

array([0., 0., 0., ..., 1., 1., 1.])

In [15]:
print(tr_pairs.shape, tr_y.shape)

(330519, 2, 16, 16) (330519,)


In [16]:
tr_pairs[0][0].shape

(16, 16)

In [17]:
tr_pairs[0][1].shape

(16, 16)

- test

In [18]:
16*16

256

In [19]:
np.array([1,2,3,4,5,6,7,8]).reshape(1,2,2,2)

array([[[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]]])

In [15]:
te_pairs = X_test.reshape(len(X_test), 2, 16, 16)
te_y  = np.array(y_test, dtype=float)

In [21]:
print(te_pairs.shape, te_y.shape)

(2183, 2, 16, 16) (2183,)


In [22]:
te_pairs[0]

array([[[1.42188190e-01, 7.30437040e-02, 6.54419840e-03, 3.47212740e-02,
         5.42317420e-03, 1.90891240e-01, 5.48556100e-02, 1.90691950e-02,
         5.31537460e-02, 0.00000000e+00, 5.68175540e-02, 5.99295230e-03,
         1.46555270e-02, 4.06781440e-02, 1.76598880e-02, 1.24414280e-02],
        [1.28487740e-02, 4.53592050e-02, 1.00350100e-01, 1.98895630e-02,
         9.20542700e-02, 3.93058300e-02, 1.24327300e-01, 3.21867800e-03,
         8.68949800e-02, 1.43282760e-02, 9.36443100e-02, 1.18206380e-01,
         1.22040810e-02, 2.52782850e-02, 9.00668600e-03, 0.00000000e+00],
        [8.05807200e-03, 9.45651700e-03, 8.39080550e-04, 0.00000000e+00,
         3.74782390e-03, 3.84895430e-02, 1.26211840e-01, 6.53332500e-02,
         1.48222810e-02, 2.08259040e-02, 1.86327680e-02, 6.20653000e-02,
         6.73222240e-02, 3.96289630e-03, 6.58821500e-02, 7.18304900e-02],
        [4.85406550e-04, 1.54494160e-02, 3.96003300e-02, 1.80409630e-02,
         1.19384915e-01, 0.00000000e+00, 1.36813

In [23]:
te_y

array([0., 0., 1., ..., 1., 1., 1.])

---
---

## ## Training

In [24]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import RMSprop
from keras import backend as K

Using TensorFlow backend.


In [25]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)


def create_pairs(x, digit_indices): # balance 하게 만들어주네???(가장 작은 class의 개수에 맞춰서)
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)




def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = input    
    x = Conv2D(32, 3, padding='same', activation='relu', 
           input_shape=( 16, 16 , 1))(x)
    x = MaxPooling2D()(x)
    #x = Dropout(0.2)(x)
    x = Conv2D(64, 3, padding='same', activation='relu')(x)
    x = MaxPooling2D()(x)
    #x = Dropout(0.2)(x)
    x = Conv2D(128, 3, padding='same', activation='relu')(x)
    x = MaxPooling2D()(x)
    #x = Dropout(0.2)(x)
    x = Conv2D(256, 3, padding='same', activation='relu')(x)
    x = MaxPooling2D()(x)
    #x = Dropout(0.2)(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    
    return Model(input, x)

def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))


In [26]:
###
input_shape = (16,16, 1)
epochs = 50

In [29]:
# network definition
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance) # input, output

# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
history = model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=500,
          epochs=epochs,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

# compute final accuracy on training and test sets
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred)
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

Epoch 1/50
662/662 [==============================] - 26s 40ms/step - loss: 0.1863 - accuracy: 0.7575 - val_loss: 0.2560 - val_accuracy: 0.6597
Epoch 2/50
662/662 [==============================] - 26s 40ms/step - loss: 0.1560 - accuracy: 0.7950 - val_loss: 0.2485 - val_accuracy: 0.6762
Epoch 3/50
662/662 [==============================] - 26s 39ms/step - loss: 0.1457 - accuracy: 0.8095 - val_loss: 0.2483 - val_accuracy: 0.6716
Epoch 4/50
662/662 [==============================] - 26s 39ms/step - loss: 0.1380 - accuracy: 0.8211 - val_loss: 0.2524 - val_accuracy: 0.6817
Epoch 5/50
662/662 [==============================] - 26s 39ms/step - loss: 0.1326 - accuracy: 0.8294 - val_loss: 0.2597 - val_accuracy: 0.6676
Epoch 6/50
662/662 [==============================] - 26s 39ms/step - loss: 0.1276 - accuracy: 0.8372 - val_loss: 0.2455 - val_accuracy: 0.6864
Epoch 7/50
662/662 [==============================] - 26s 39ms/step - loss: 0.1235 - accuracy: 0.8429 - val_loss: 0.2375 - val_accuracy:

In [34]:
( 1*( y_pred.ravel() < 0.5 ) )

array([0, 0, 1, ..., 1, 1, 1])

In [33]:
( 1*( y_pred.ravel() < 0.5 ) ).shape

(2183,)

In [37]:
te_y.shape

(2183,)

In [35]:
y_test_dict

{40648: array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0]),
 66052: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]),
 37783: array([1, 1, 0, 0, 0]),
 59761: array([0, 0, 0, 1, 1]),
 16639: array([1, 1, 1, 1]),
 17958: array([1, 1, 1, 1, 1]),
 10918: array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 26542: array([1, 1, 1, 1]),
 71651: array([0, 0, 0, 1, 1]),
 45696: array([0, 1]),
 4253: array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]),
 63461: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]),
 572: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]),
 66460: array([0]),
 70828: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0]),
 34741: array([0, 0, 0, 0, 0, 0, 

In [50]:
len(y_test_dict.keys())

100

In [39]:
y_pred

array([[1.562553  ],
       [1.5733085 ],
       [0.18377545],
       ...,
       [0.16946608],
       [0.04108986],
       [0.14604653]], dtype=float32)

In [48]:
y_pred.ravel()

array([1.562553  , 1.5733085 , 0.18377545, ..., 0.16946608, 0.04108986,
       0.14604653], dtype=float32)

In [42]:
np.save('cnn_predicted', y_pred.ravel())

In [ ]:
cnn_predicted.npy

In [17]:
y_pred = np.load('cnn_predicted.npy')

In [52]:
te_y.shape

(2183,)

In [54]:
( 1*( y_pred < 0.5 ) ).shape

(2183, 1)

In [55]:
y_pred.ravel()[0:5]

array([1.562553  , 1.5733085 , 0.18377545, 0.46056923, 0.82140964],
      dtype=float32)

In [57]:
list(y_test_dict.keys())[0]

40648

In [59]:
len( y_test_dict[40648] )

10

In [21]:
from sklearn import metrics

precision_100 = []
recall_100 = []

grp_pred_w_key = {}
grp_label_w_key = {}

test_label = te_y
predicted = ( 1*( y_pred.ravel() < 0.5 ) ) # 초과를 봐야해 미만을 봐야해?

#start test key
start_test_key = list(y_test_dict.keys())[0]
#init start index
start_index = 0

for test_key in y_test_dict.keys(): 
    print(test_key)
    if test_key == start_test_key:
        start_index = 0
    
    test_obj_grp_size = len( y_test_dict[test_key] )
    grp_pred = predicted[start_index : start_index + test_obj_grp_size]
    grp_label = test_label[start_index : start_index +test_obj_grp_size]
    # 저장
    grp_pred_w_key[test_key] = grp_pred
    grp_label_w_key[test_key] = grp_label
    ####
    
    print(start_index, test_obj_grp_size)
    
    start_index += test_obj_grp_size
    
    print('label_size : {}'.format(np.sum( grp_label )), 
          'prediction_size : {}'.format(np.sum( grp_pred )) )
    
    # calculate precision / recall
    precision_ = metrics.precision_score( grp_label , grp_pred )
    recall_ = metrics.recall_score( grp_label , grp_pred )
    
    # print
    print( precision_ )
    print( recall_ )
    print('\n')
    
    # append precision / recall
    precision_100.append( precision_ )
    recall_100.append( recall_ )

40648
0 10
label_size : 2.0 prediction_size : 5
0.4
1.0


66052
10 17
label_size : 2.0 prediction_size : 2
1.0
1.0


37783
27 5
label_size : 2.0 prediction_size : 3
0.6666666666666666
1.0


59761
32 5
label_size : 2.0 prediction_size : 2
1.0
1.0


16639
37 4
label_size : 4.0 prediction_size : 4
1.0
1.0


17958
41 5
label_size : 5.0 prediction_size : 5
1.0
1.0


10918
46 18
label_size : 5.0 prediction_size : 15
0.13333333333333333
0.4


26542
64 4
label_size : 4.0 prediction_size : 4
1.0
1.0


71651
68 5
label_size : 2.0 prediction_size : 2
1.0
1.0


45696
73 2
label_size : 1.0 prediction_size : 1
1.0
1.0


4253
75 12
label_size : 8.0 prediction_size : 8
1.0
1.0


63461
87 96
label_size : 2.0 prediction_size : 61
0.03278688524590164
1.0


572
183 17
label_size : 14.0 prediction_size : 17
0.8235294117647058
1.0


66460
200 1
label_size : 0.0 prediction_size : 0
0.0
0.0


70828
201 24
label_size : 1.0 prediction_size : 6
0.0
0.0


34741
225 170
label_size : 4.0 prediction_size : 35
0.1142

In [22]:
np.mean(precision_100)

0.7470528333613038

In [23]:
np.mean(recall_100)

0.9045833333333334

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

